## BERT fine tune NER

**Bert** is a transformers model pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labeling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts.

**conll2003:**
The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on four types of named entities: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups.

The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q datasets
!pip install -q seqeval

• 0 means the word doesn't correspond to any entity. 
• B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity. 
• B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity. 
• B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity. 
• B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity. 

In [1]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer, DataCollator
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = load_dataset("conll2003")

Learning POS Tagging & Chunking in NLP: https://medium.com/greyatom/learning-pos-tagging-chunking-in-nlp-85f7f811a8cb

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
dataset.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

In [5]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [6]:
idx = 0

print(dataset['train'][idx])

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [7]:
dataset['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [13]:
dataset['train'].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [14]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [20]:
sample = dataset['train'][0]
print(sample)

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


**tokenizer(input):** When you use tokenizer(input), you're essentially tokenizing the input text. This means breaking down the input text into individual tokens, which could be words, subwords, or characters, depending on the tokenizer used. This operation returns a list of tokens.
tokenizer.encode(input):

**tokenizer.encode(input):** goes one step further than just tokenizing. It not only tokenizes the input but also converts those tokens into corresponding numerical IDs, often referred to as token IDs or input IDs. These numerical IDs are what the model actually operates on. This operation returns a list of token IDs.

In [35]:
"""
Returns only input_ids but tokenizer(input) returns: input_ids, token_type_ids, attention_mask
"""
print(tokenizer.encode(' '.join(sample['tokens']), add_special_tokens=True))

[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]


In [23]:
tokenized_sample = tokenizer(sample['tokens'], is_split_into_words=True)

In [25]:
tokenized_sample

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_sample['input_ids'])
print(tokens)

['[CLS]', 'eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.', '[SEP]']


In [38]:
word_ids = tokenized_sample.word_ids() # return list of mapping the tokens, to their actual word in the initial sentence
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [39]:
len(sample['ner_tags']), len(tokenized_sample['input_ids'])

(9, 11)

Size between the ner_tags in the sample text and the tokenized input_ids from the sample (One is larger)
-> **PROBLEM OF SUB-TOKEN**
WHY? because transformers are often trainer on sub_words tokenizers, so even if we give splitted inputs, it can be splitted again by the tokenizer. And also because some special tokens maybe added like ('[CLS]', '[SEP]')
-> **Solution**
Get based on `.word_ids()` method because it sets the special tokens to `None`. We will set the labels for all special tokens to $-100$ because it gets ignored by pytorch during training. And all other tokens to the word they come from.  

In [41]:
def tokenize_and_align_labels(example, label_all_tokens = True):
    """
    This function will do two things:
        1. Set -100 as the label for special tokens
        2. Mask the sub-words representation after the first sub-word
    Parameters:
        example (str): the dataset
        label_all_tokens (bool): define if we will apply tokenization
    return: 
    """
    
    tokenized_example = tokenizer(example['tokens'], is_split_into_words=True, truncation=True)
    labels = []
    for i, label in enumerate(example['ner_tags']):
        word_ids = tokenized_example.word_ids(batch_index=i)
        prev_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                # common scenario
                label_ids.append(label[word_idx])
            else:
                # take care of sub-words
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            prev_word_idx = word_idx    
        labels.append(label_ids)
    tokenized_example['labels'] = labels
    return tokenized_example
    

In [43]:
idx = 6
print(tokenize_and_align_labels(dataset['train'][idx:idx+1]))

{'input_ids': [[101, 2002, 2056, 2582, 4045, 2817, 2001, 3223, 1998, 2065, 2009, 2001, 2179, 2008, 2895, 2001, 2734, 2009, 2323, 2022, 2579, 2011, 1996, 2647, 2586, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, -100]]}


`labels` is added, represente the aligned values